<a href="https://colab.research.google.com/github/buixuanthanh96/CS114.K21/blob/master/Sacarsm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sinh viên thực hiện: Bùi Xuân Thành
MSSV:14520830

1. **Mô tả bài toán và thu thập dữ liệu**:
- Sacarsm detection là bài toán phát hiện trào phúng, châm biếm.
- Dữ liệu bài toán được thu tập từ 2 nguồn trang web tin tức là: The Onion là một tờ báo trào phúng của Mỹ thường viết những bài báo mang tính châm biếm, trào phúng, tường thuật các tin tức quốc tế và HuffPost là một trang tin tức trực tuyến và log về tin tức và ý kiến của Mỹ.
- Link nguồn lấy dữ liệu:  https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/download
- Tập dữ liệu có tổng cộng 26709 mẫu trong đó có 11724 mẫu châm biếm và 14985 mẫu không châm biếm.
- Mỗi mẫu dữ liệu gồn 3 thuộc tính :
•	article_link: Liên kết với bài viết tin tức.
•	headline: tiêu đề bài báo.
•	is_sarcastic : bài báo có châm biếm hay không(1 là châm chiếm và 0 không châm biếm).


2. **Thu thập 2000 Headlines mới**
- Dữ liệu mới gồm 1000 Headlines được thu thập từ trang báo châm biếm theonion và 1000 Headlines được thu thập từ trang báo huffpost

3. **Mô tả cách xử lý dữ liệu, feature engineering trên dataset đã cho**
- Dữ liệu đầu tiên được xử lý loại bỏ dấu câu và số.
- Sau đó được xử lý loại bỏ stopword
- Kế tiếp tách Headline thành các từ và chuyển các từ về dạng nguyên mẫu, kí tự thường
- Feature Engineering: dùng TF-IDF Vectors as features để đưa dữ liệu dạng văn bản đã được xử lý về dạng vector thuộc tính có dạng số học.


4. **Mô tả quá trình chọn model, học và fine tuning**
- Chọn Naive Bayes vì thường được sử dụng trong các bài toán Text Classification có khả năng học tập và dự đoán hiệu quả cao, nhanh và có khả năng mở rộng cao (hoạt động tốt với dữ liệu nhiều chiều). 
- Dữ liệu được chia thành train/test theo tỉ lệ 80/20.
fine tuning : alpha=1.0, fit_prior=True, class_prior=None
- Chạy SVM để so sánh thấy Naive Bayes nhỉnh hơn so với svm

6.**Đối chiếu performance của model trên dataset đã cho và trên 2000 headine mới. Nêu nhận xét của nhóm về bài toán này.**
- Accuracy chạy trên dữ liệu cũ của cả 2 model đều lớn hơn accuracy chạy trên bộ dữ liệu test mới.
=> dữ liệu thu thập ảnh hưởng rất lớn đến performance của model
 

In [1]:
#Lấy dữ liệu từ git xuống
#Sarcasm_Headlines_Dataset.json : dữ liệu mẫu có sẵn
# new_Sarcasm_Headlines_Dataset.json : là 2000 headlines mới tự thu thập
!git clone https://github.com/buixuanthanh96/SacarsmDetection.git

Cloning into 'SacarsmDetection'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#xử lý dữ liệu có sẵn
import pandas as pd
import string, re, nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def getPOS(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()#xác định loại từ
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) #trả về dưới dạng danh từ

path = "SacarsmDetection/input/"
data = pd.read_json(path + 'Sarcasm_Headlines_Dataset.json', lines=True)
data = data.drop(columns = ["article_link"])

lemmatizer = WordNetLemmatizer()
headlinesPP = []
for index, row in data.iterrows():
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.punctuation)) #Loại bỏ dấu câu
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.digits)) #Loại bỏ kí tự số
    row["headline"] = ' '.join([word for word in row["headline"].split() if word not in (stopwords.words('english'))])#loại bỏ stopwords
    tokenList = re.sub("[^\S]", " ",  row["headline"]).split() #tách chuỗi thành các từ
    #tokenList = nltk.word_tokenize(row["headline"])
    for i in range(0, len(tokenList)): #lemmatization
        tokenList[i] = lemmatizer.lemmatize(tokenList[i], getPOS(tokenList[i]))#chuyển từ về dạng nguyên mẫu
    headlinesPP.append(tokenList)
dataPP = pd.DataFrame(data = {'headline': headlinesPP, 'is_sarcastic': data["is_sarcastic"]} )
dataPP.to_csv(path+'dataPostProcessed.csv')

In [4]:
#xử lý 2000 mẫu dữ liệu tự thu thập
import pandas as pd
import string, re, nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def getPOS(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()#xác định loại từ
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) #trả về dưới dạng danh từ

path = "SacarsmDetection/input/"
data = pd.read_json(path + 'new_Sarcasm_Headlines_Dataset.json', lines=True)
data = data.drop(columns = ["article_link"])

lemmatizer = WordNetLemmatizer()
headlinesPP = []
for index, row in data.iterrows():
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.punctuation)) #Loại bỏ dấu câu
    row["headline"] = row["headline"].translate(str.maketrans('', '', string.digits)) #Loại bỏ kí tự số
    row["headline"] = ' '.join([word for word in row["headline"].split() if word not in (stopwords.words('english'))])#loại bỏ stopwords
    tokenList = re.sub("[^\S]", " ",  row["headline"]).split() #tách chuỗi thành các từ
    #tokenList = nltk.word_tokenize(row["headline"])
    for i in range(0, len(tokenList)): #lemmatization
        tokenList[i] = lemmatizer.lemmatize(tokenList[i], getPOS(tokenList[i]))#chuyển từ về dạng nguyên mẫu
    headlinesPP.append(tokenList)
dataPP = pd.DataFrame(data = {'headline': headlinesPP, 'is_sarcastic': data["is_sarcastic"]} )
dataPP.to_csv(path+'new_dataPostProcessed.csv')

In [55]:
#Chạy model Naive Bayes
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

path = "SacarsmDetection/input/"
data = pd.read_csv(path + 'dataPostProcessed.csv', index_col=0)
X = data['headline']#.append(df['headline'])
Y = data['is_sarcastic']#.append(df['is_sarcastic'])

new_input = pd.read_csv(path + 'new_dataPostProcessed.csv', index_col=0)
X_new_test = new_input['headline']#.append(df['headline'])
Y_new_test = new_input['is_sarcastic']#.append(df['is_sarcastic'])



xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.2)
print("*** Số mẫu train: "+str(len(xTrain))+ ", test: "+str(len(xTest)))
classifier = Pipeline([('tfidf', TfidfVectorizer()), ('classifier', MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None))])
    
classifier.fit(xTrain, yTrain)
print("###### Naive Bayes #########")
prediction = classifier.predict(xTest)
prediction_new = classifier.predict(X_new_test)
#print(classification_report(yTest, prediction))
print("accuracy khi test với dữ liệu cũ: ",metrics.accuracy_score(yTest, prediction))
print("accuracy khi test với 2000 dữ liệu mới: ",metrics.accuracy_score(Y_new_test, prediction_new))


*** Số mẫu train: 21367, test: 5342
###### Naive Bayes #########
accuracy khi test với dữ liệu cũ:  0.7905278921752152
accuracy khi test với 2000 dữ liệu mới:  0.655


In [13]:
#Chạy model SVM
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

path = "SacarsmDetection/input/"
data2 = pd.read_csv(path + 'dataPostProcessed.csv', index_col=0)
X = data2['headline']
Y = data2['is_sarcastic']

new_input = pd.read_csv(path + 'new_dataPostProcessed.csv', index_col=0)
X_new_test = new_input['headline']
Y_new_test = new_input['is_sarcastic']



xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size=0.2)
print("*** Số mẫu train: "+str(len(xTrain))+ ", test: "+str(len(xTest)))

classifier2 = Pipeline([('tfidf', TfidfVectorizer()), ('classifier', LinearSVC())])
 
classifier2.fit(xTrain, yTrain)
print("###### LinearSVC #########")
prediction = classifier2.predict(xTest)
prediction_new = classifier2.predict(X_new_test)
#print(classification_report(yTest, prediction))
print("accuracy khi test với dữ liệu cũ: ",metrics.accuracy_score(yTest, prediction))
print("accuracy khi test với 2000 dữ liệu mới: ",metrics.accuracy_score(Y_new_test, prediction_new))

*** Số mẫu train: 21367, test: 5342
###### LinearSVC #########
accuracy khi test với dữ liệu cũ:  0.7813552976413328
accuracy khi test với 2000 dữ liệu mới:  0.671


In [46]:
print("Nhập vào headline:")
headline = str(input())
headline = headline.translate(str.maketrans('', '', string.punctuation)) #Loại bỏ dấu câu
headline = headline.translate(str.maketrans('', '', string.digits)) #Loại bỏ kí tự số
headline = ' '.join([word for word in headline.split() if word not in (stopwords.words('english'))])#loại bỏ stopwords
tokenList = re.sub("[^\S]", " ",  headline).split() #tách chuỗi thành các từ
#tokenList = nltk.word_tokenize(row["headline"])
for i in range(0, len(tokenList)): #lemmatization
    tokenList[i] = lemmatizer.lemmatize(tokenList[i], getPOS(tokenList[i]))
prediction1 = classifier.predict( [headline])
print("Sacarsm detection : ", prediction1)

Nhập vào headline:
Trump Inadvertently Saves Life Of Yemeni Family After Appropriating Pentagon Money For Border Wall
Sacarsm detection :  [1]
